In [29]:
import torch
import torch.nn as nn
from loss import Loss
from dataset import *
import model
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import os

In [2]:
train_data = Fuck(img_dir="train_images", csv_file="train.csv", transform=data_transform)

In [3]:
img, label = train_data.__getitem__(10)
img.shape, label.shape

(torch.Size([3, 224, 224]), torch.Size([6]))

In [37]:
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=0)
x, y = next(iter(train_data_loader))

In [6]:
x.shape, y.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32, 6]))

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
class block(nn.Module):
    def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super().__init__()
        self.expansion = 4
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False,),
            nn.BatchNorm2d(intermediate_channels),
            nn.ReLU(),
            nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False,),
            nn.BatchNorm2d(intermediate_channels),
            nn.ReLU(),
            nn.Conv2d(intermediate_channels, intermediate_channels * self.expansion, kernel_size=1, stride=1, padding=0, bias=False,),
            nn.BatchNorm2d(intermediate_channels * self.expansion)
        )
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)

        return x

In [9]:
thismodel = model.ResNet(block, [3, 4, 6, 3], 3, 6).to(device)

In [10]:
thismodel.load_state_dict(torch.load(f='models/fuck_more.pth'))

<All keys matched successfully>

In [11]:
loss_fn = Loss()
optimizer = torch.optim.Adam(params=thismodel.parameters(), lr=0.001)

In [12]:
torch.cuda.is_available()

True

In [13]:
thismodel(x.to(device))

tensor([[  4.3757,  -4.5043,  -6.3846,  -7.5389,  -6.6134,  -6.4016],
        [ -2.3937,  -3.1327,  -3.8957,  -3.1797,  -4.9305,   1.6236],
        [  3.3598,  -3.3693,  -6.4653,  -7.5137,  -5.7472,  -5.4853],
        [ -6.6627,  -1.1818,   0.0671,  -1.6143,  -2.2725,  -7.0922],
        [  1.3986,  -3.0707,  -4.7132,  -7.3536,  -5.9112,  -1.9952],
        [  2.7750,  -2.4544,  -6.5787,  -7.9448,  -5.0769,  -7.6685],
        [  4.3262,  -4.7729,  -6.2370,  -7.9692,  -4.1117,  -8.1967],
        [ -1.7568,   0.8336,  -3.2993,  -4.6051,  -4.7038,  -4.7773],
        [ -7.7704,   2.4325,  -2.5118,  -2.0963,  -5.6295,  -7.1946],
        [ -9.7590,  -1.8055,   1.3308,  -1.2829,  -4.3230,  -8.0936],
        [ -0.5202,  -0.3325,  -4.5626,  -4.4172,  -5.4405,  -2.9558],
        [ -3.3391,   3.2457,  -6.7366,  -4.6384,  -5.0567,  -6.9695],
        [  4.8294,  -4.6004,  -6.5730,  -7.5255,  -6.3299,  -6.4973],
        [ -6.5948,  -3.3503,   0.6898,  -2.8001,  -1.7983,  -9.1929],
        [ -8.0796,  

In [16]:
epochs = 5

for epoch in tqdm(range(epochs)):
    thismodel.train()

    train_loss = 0
    train_acc = 0
    correct = 0
    total_sample = 0
    for batch_idx, (X, y) in enumerate(train_data_loader):
        X = X.to(device)
        y = y.to(device)

        y_logits = thismodel(X).squeeze()
        loss = loss_fn(y_logits, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_preds = torch.round(torch.sigmoid(y_logits))
        correct += torch.eq(y_preds, y).sum().item()
        # train_acc = (correct / len(y_preds)) * 100
        total_sample += y.size(0)

    train_loss /= len(train_data_loader)
    train_acc = correct / total_sample
    
    print(f"epoch: {epoch}")
    print(f"train_loss: {train_loss} | train_acc: {train_acc}")

  0%|          | 0/5 [00:00<?, ?it/s]

epoch: 0
train_loss: 0.17181873941349943 | train_acc: 5.597091026191499
epoch: 1
train_loss: 0.15470673527255474 | train_acc: 5.632406612279948
epoch: 2
train_loss: 0.14511717426531515 | train_acc: 5.655002146844139
epoch: 3
train_loss: 0.13714026542792598 | train_acc: 5.6762559038213825
epoch: 4
train_loss: 0.13392288322156332 | train_acc: 5.6835551738943755


In [17]:
from pathlib import Path

In [35]:
model_path = Path('models')
model_path.mkdir(parents=True, exist_ok=True)

model_name = 'fuck.pth'
model_save_path = model_path/model_name

torch.save(obj=thismodel.state_dict(),f=model_save_path)

In [26]:
thismodel.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-2.7750e-02,  1.0674e-01, -3.8666e-02,  ..., -3.3669e-02,
                          2.7996e-02,  1.7497e-02],
                        [ 1.2600e-01,  4.1770e-02,  1.3156e-01,  ...,  9.2351e-02,
                          1.1081e-01,  1.5138e-01],
                        [-3.7164e-02,  7.0159e-02,  5.0023e-02,  ...,  5.7537e-02,
                          5.9279e-02,  9.3964e-02],
                        ...,
                        [ 8.1599e-03,  7.4101e-02,  5.3288e-02,  ...,  9.5915e-02,
                          1.2145e-01,  7.6512e-02],
                        [ 7.6148e-02, -2.0613e-02, -1.2064e-02,  ...,  7.0028e-02,
                          6.3146e-02,  7.4276e-02],
                        [ 8.4228e-03,  1.9180e-02,  8.6778e-02,  ...,  4.1781e-02,
                          5.4493e-02,  1.3773e-01]],
              
                       [[ 8.4177e-02, -1.2120e-03, -4.1360e-02,  ...,  7.4108e-02,
                         -4.2412

In [48]:
model_path = Path('models')
model_path.mkdir(parents=True, exist_ok=True)

model_name = 'fuck_yeah.pth'
model_save_path = model_path/model_name

torch.save(obj=thismodel.state_dict(),f=model_save_path)

In [34]:
img, label = train_data.__getitem__()
img, label

(tensor([[[0.1608, 0.1569, 0.1608,  ..., 0.4667, 0.4824, 0.4980],
          [0.1804, 0.1725, 0.1765,  ..., 0.4706, 0.4824, 0.5020],
          [0.2039, 0.1765, 0.1804,  ..., 0.4745, 0.4863, 0.5020],
          ...,
          [0.2235, 0.1961, 0.2078,  ..., 0.4510, 0.4588, 0.4667],
          [0.1961, 0.1961, 0.3137,  ..., 0.4196, 0.4157, 0.4235],
          [0.2000, 0.3137, 0.4275,  ..., 0.4157, 0.3961, 0.4000]],
 
         [[0.3725, 0.3569, 0.3569,  ..., 0.6980, 0.7059, 0.7216],
          [0.3843, 0.3725, 0.3725,  ..., 0.7059, 0.7059, 0.7255],
          [0.4000, 0.3725, 0.3686,  ..., 0.7098, 0.7137, 0.7294],
          ...,
          [0.3843, 0.3608, 0.3608,  ..., 0.6824, 0.6824, 0.6784],
          [0.3569, 0.3529, 0.4510,  ..., 0.6667, 0.6588, 0.6549],
          [0.3608, 0.4627, 0.5647,  ..., 0.6706, 0.6471, 0.6392]],
 
         [[0.2118, 0.1725, 0.1725,  ..., 0.6157, 0.6314, 0.6471],
          [0.2275, 0.2000, 0.1961,  ..., 0.6235, 0.6314, 0.6510],
          [0.2510, 0.2196, 0.2078,  ...,

In [35]:
y = thismodel(img.unsqueeze(0).to(device))

In [36]:
y_preds = torch.sigmoid(y)
y_preds

tensor([[3.1760e-07, 6.0792e-03, 6.7162e-03, 1.9617e-02, 6.0228e-03, 5.2880e-07]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [38]:
x, y

(tensor([[[[0.3961, 0.6196, 0.7569,  ..., 1.0000, 1.0000, 1.0000],
           [0.4000, 0.6275, 0.7765,  ..., 1.0000, 1.0000, 1.0000],
           [0.4118, 0.6314, 0.8039,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [0.5373, 0.4157, 0.3255,  ..., 1.0000, 1.0000, 1.0000],
           [0.4431, 0.3922, 0.3216,  ..., 0.9843, 0.9882, 0.9922],
           [0.4235, 0.3765, 0.3255,  ..., 0.9412, 0.9490, 0.9490]],
 
          [[0.4784, 0.7333, 0.8784,  ..., 1.0000, 1.0000, 1.0000],
           [0.4824, 0.7373, 0.9020,  ..., 1.0000, 1.0000, 1.0000],
           [0.4941, 0.7490, 0.9412,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [0.6902, 0.5765, 0.4863,  ..., 1.0000, 1.0000, 1.0000],
           [0.5882, 0.5412, 0.4745,  ..., 1.0000, 1.0000, 1.0000],
           [0.5647, 0.5137, 0.4627,  ..., 1.0000, 1.0000, 1.0000]],
 
          [[0.1098, 0.2431, 0.3412,  ..., 0.9843, 0.9529, 0.8941],
           [0.1216, 0.2627, 0.3686,  ..., 0.9412, 0.8824, 0.8275],
           [0.1373, 0.27

In [43]:
x = x.to(device)
y = y.to(device)

preds = thismodel(x)
preds = torch.sigmoid(preds)
preds

tensor([[9.9925e-01, 7.3949e-04, 6.4534e-04, 3.7528e-04, 8.4819e-05, 1.0223e-05],
        [1.5817e-03, 8.4945e-01, 1.8686e-01, 1.0666e-01, 1.1148e-02, 1.7858e-02],
        [2.7296e-08, 6.5799e-04, 9.8944e-01, 2.4958e-03, 8.8984e-03, 9.1289e-08],
        [5.7544e-03, 7.5646e-03, 8.6175e-01, 3.9002e-02, 4.6279e-02, 1.1240e-02],
        [6.3840e-05, 9.8923e-01, 6.4191e-03, 2.2688e-02, 9.2093e-04, 1.4192e-04],
        [1.6540e-08, 9.9862e-01, 6.9071e-03, 2.3066e-03, 8.4550e-05, 2.0634e-07],
        [9.9748e-01, 3.5512e-03, 8.5757e-04, 3.9038e-04, 4.1957e-04, 9.7624e-05],
        [4.1928e-03, 1.6195e-03, 3.9292e-04, 4.3409e-03, 5.1552e-05, 9.9652e-01],
        [4.6055e-04, 9.9847e-01, 1.4595e-04, 1.8988e-03, 2.4316e-04, 2.4619e-05],
        [9.8586e-01, 8.0622e-03, 1.2278e-03, 9.6892e-04, 2.6892e-04, 9.8584e-03],
        [9.9634e-01, 7.0552e-03, 3.2896e-04, 2.6353e-04, 1.7646e-04, 7.7765e-05],
        [9.8452e-01, 4.3526e-02, 1.3301e-03, 1.2250e-03, 1.5849e-03, 3.3396e-04],
        [7.3367e

In [44]:
preds = torch.round(preds)
preds

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0

In [45]:
num_correct = 0
total = y.shape[0]

for i in range(y.shape[0]):
    count = 0
    for j in range(y.shape[1]):
        if preds[i, j] == y[i, j]:
            count += 1

    if count == 6: 
        num_correct += 1

num_correct / total

0.875

In [46]:
total

32